In [88]:
import time
from random import randint
import json

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup

In [77]:
# Selenium driver options
options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; WOW64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4864.133 Safari/537.36")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

In [ ]:
# Get all smartphone links
link = "https://market.yandex.ru/catalog--smartfony/26893750/list?hid=91491&local-offers-first=0"
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

all_links = []

try:
    i = 1
    while True:
        # Opening the page
        driver.get(link + f"&page={i}")
        
        # we wait, because otherwise we will be discovered :)
        time.sleep(randint(10, 15))
        
        # Scroll down to load all items
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(0.1)
        
        # looking for all blocks that contain content about the phone
        soup = BeautifulSoup(driver.page_source, "lxml")
        div_blocks = soup.find_all("div", {"data-tid": "dc53a9af 69501d18"}, class_="_3KhA2 _1jTgr",)
        
        # if empty, exit
        if not div_block:
            break
        
        # in each block found, we go down to the link that is 
        # in the header (you can also take the link from the picture)
        for div_block in div_blocks[1:]:
            title_block = div_block.find_next("h3", class_="_2UHry _2vVOc")
            product_link = "https://market.yandex.ru" + title_block.a.get("href")
            all_links.append(product_link)
        
        i += 1
    
except Exception as e:
    print(e)
    
finally:
    driver.close()
    driver.quit()

In [ ]:
# We use the manager to automatically download the driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

comments = []

try:
    for link in all_links:
        # Go to the product page
        driver.get(link)
        time.sleep(10)

        # Go to the reviews page
        soup = BeautifulSoup(driver.page_source, "lxml")
        link_reviews = soup.find("a", class_="_2f75n _2J5l3 _15qW8 cia-cs").get("href")

        i = 1
        while True:
            # go to the next page
            time.sleep(randint(10, 15))
            driver.get("https://market.yandex.ru" + link_reviews + f"&page={i}")
            soup = BeautifulSoup(driver.page_source, "lxml")
            
            # We are looking for all blocks with user comments
            divs_reviews = soup.find_all("div", {"data-zone-name": "product-review"})
            
            # if empty, exit
            if not divs_reviews:
                break

            for div_review in divs_reviews:
                # We get the rating that the user put
                rate = div_review.find_next("div", class_="_3iy4z _3DD8b autotest-RatingStars").get("data-rate")
                
                # we get the block in which the user reviews are located
                dd_reviews = div_review.find_all("dd", class_="_1yULR jwsQ_")
                
                # If such a block is not found, then we are looking for another class
                if not dd_reviews:
                    dd_reviews = div_review.find_all("dd", class_="_1yULR")
                
                # merge all comments into one
                review = ' '.join(block.string for block in dd_reviews)
                
                # Adding the generated block to the list
                comments.append({
                    "comment": review,
                    "rate": rate
                })

            i += 1
    
    # Write data to file like json
    with open("dataset.json", "w") as file:
        json.dump(comments, file)
    

except Exception as e:
    print(e)

finally:
    driver.close()
    driver.quit()